In [2]:
import os
import sys
maindir = os.getcwd()
sys.path.append(maindir+"/src")

In [3]:
import pickle
import torch
import numpy as np
import matplotlib.pyplot as plt

from preprocessing import data_processing, compute_anomalies, \
                            compute_forced_response, compute_variance, \
                            merge_runs, stack_runs, numpy_to_torch, standardize, build_training_and_test_sets,\
                            build_training_and_test_sets_stacked

from plot_tools import plot_gt_vs_pred, animation_gt_vs_pred
from leave_one_out import leave_one_out_single, leave_one_out_procedure
from cross_validation import cross_validation_procedure

In [4]:
############### Load climate model raw data for SST
with open('data/ssp585_time_series.pkl', 'rb') as f:
    data = pickle.load(f)

###################### Load longitude and latitude 
with open('data/lon.npy', 'rb') as f:
    lon = np.load(f)

with open('data/lat.npy', 'rb') as f:
    lat = np.load(f)

# define grid (+ croping for latitude > 60)
lat_grid, lon_grid = np.meshgrid(lat[lat<=60], lon, indexing='ij')

lat_size = lat_grid.shape[0]
lon_size = lon_grid.shape[1]

In [5]:
# define pytorch precision
dtype = torch.float32

data_processed, notnan_idx, nan_idx = data_processing(data, lon, lat,max_models=100)
x = compute_anomalies(data_processed, lon_size, lat_size, nan_idx, time_period=33)
y = compute_forced_response(data_processed, lon_size, lat_size, nan_idx, time_period=33)
vars = compute_variance(x, lon_size, lat_size, nan_idx, time_period=33)

# convert numpy arrays to pytorch 
x, y, vars = numpy_to_torch(x,y,vars)

# standardize data 
x, y = standardize(x,y,vars)

# stack runs for each model
x, y, vars = stack_runs(x,y,vars,time_period=33,lon_size=lon_size,lat_size=lat_size,dtype=dtype)

# stack runs for each model
x_merged, y_merged, vars_merged = merge_runs(x,y,vars)

/home/vcohen/cope/src/preprocessing.py:109: RuntimeWarning: Mean of empty slice
  mean_ref_ensemble = np.nanmean(y_tmp,axis=1)
/home/vcohen/cope/src/preprocessing.py:110: RuntimeWarning: Mean of empty slice
  mean_ref_ensemble = np.nanmean(mean_ref_ensemble,axis=0)
/home/vcohen/cope/src/preprocessing.py:152: RuntimeWarning: Mean of empty slice
  mean_spatial_ensemble = np.nanmean(y_tmp,axis=0)
/home/vcohen/cope/src/preprocessing.py:156: RuntimeWarning: Mean of empty slice
  data_forced_response[m][r] = mean_spatial_ensemble - np.nanmean(mean_spatial_ensemble,axis=0)


In [6]:
m0 = 'IPSL-CM6A-LR'

training_models, x_train, y_train, x_test, y_test = build_training_and_test_sets_stacked(m0,x,y,vars,lon_size,lat_size,time_period=33,dtype=dtype)

# Construct variational autoencoder 

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim

class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim):
        super(Encoder, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.hidden_to_mean = nn.Linear(hidden_dim, latent_dim)
        self.hidden_to_logvar = nn.Linear(hidden_dim, latent_dim)

    def forward(self, x):
        _, (h, _) = self.lstm(x)
        h = h[-1]
        mean = self.hidden_to_mean(h)
        logvar = self.hidden_to_logvar(h)
        return mean, logvar

class Decoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim, output_dim):
        super(Decoder, self).__init__()
        self.latent_to_hidden = nn.Linear(latent_dim, hidden_dim)
        self.lstm = nn.LSTM(hidden_dim, hidden_dim, batch_first=True)
        self.hidden_to_output = nn.Linear(hidden_dim, output_dim)

    def forward(self, z, seq_len):
        h = self.latent_to_hidden(z).unsqueeze(1).repeat(1, seq_len, 1)
        out, _ = self.lstm(h)
        out = self.hidden_to_output(out)
        return out

class TimeVAE(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim, output_dim):
        super(TimeVAE, self).__init__()
        self.encoder = Encoder(input_dim, hidden_dim, latent_dim)
        self.decoder = Decoder(latent_dim, hidden_dim, output_dim)

    def reparameterize(self, mean, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mean + eps * std

    def forward(self, x):
        mean, logvar = self.encoder(x)
        z = self.reparameterize(mean, logvar)
        recon_x = self.decoder(z, x.size(1))
        return recon_x, mean, logvar

def loss_function(recon_x, x, mean, logvar):
    recon_loss = nn.MSELoss()(recon_x, x)
    # recon_loss = torch.sum((recon_loss - x)**2)  
    kld_loss = -0.5 * torch.sum(1 + logvar - mean.pow(2) - logvar.exp())
    return recon_loss 

# Example usage
input_dim = len(notnan_idx)  # Number of features in the time series
hidden_dim = 64
latent_dim = 16
output_dim = len(notnan_idx)  # Same as input_dim

# number of epochs
nbEpochs = 300


model = TimeVAE(input_dim, hidden_dim, latent_dim, output_dim)
optimizer = optim.SGD(model.parameters(), lr=1e-3)


model.train()
for epoch in range(nbEpochs):
    optimizer.zero_grad()
    recon_x, mean, logvar = model(x_train[:,:,notnan_idx])
    loss = loss_function(recon_x, y_train[:,:,notnan_idx], mean, logvar)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch + 1}, Loss: {loss.item()}")


Epoch 1, Loss: 0.4315734803676605
Epoch 2, Loss: 0.4315020442008972
Epoch 3, Loss: 0.4315658211708069
Epoch 4, Loss: 0.4315960109233856
Epoch 5, Loss: 0.4312801957130432
Epoch 6, Loss: 0.4313614070415497
Epoch 7, Loss: 0.43144547939300537
Epoch 8, Loss: 0.43132346868515015
Epoch 9, Loss: 0.4314955472946167
Epoch 10, Loss: 0.4313679337501526
Epoch 11, Loss: 0.43152540922164917
Epoch 12, Loss: 0.4313771724700928
Epoch 13, Loss: 0.43150803446769714
Epoch 14, Loss: 0.4315594732761383
Epoch 15, Loss: 0.4314994812011719
Epoch 16, Loss: 0.4313957095146179
Epoch 17, Loss: 0.4312954545021057
Epoch 18, Loss: 0.43146273493766785
Epoch 19, Loss: 0.43134909868240356
Epoch 20, Loss: 0.43134620785713196
Epoch 21, Loss: 0.4313431680202484
Epoch 22, Loss: 0.43147629499435425
Epoch 23, Loss: 0.43158283829689026
Epoch 24, Loss: 0.4314485490322113
Epoch 25, Loss: 0.4314967095851898
Epoch 26, Loss: 0.4316745698451996
Epoch 27, Loss: 0.4313768148422241
Epoch 28, Loss: 0.4316217601299286
Epoch 29, Loss: 0.43

In [8]:
y_pred_vae = model(x_test[:,:,notnan_idx])[0]


In [9]:
from algorithms import ridge_regression, ridge_regression_low_rank, low_rank_projection, \
                        prediction, train_robust_weights_model, compute_weights

In [10]:
lambda_tmp = 10.0

# compute the big matrix X and Y
training_models, x_train_merged, y_train_merged, x_test_merged, y_test_merged = build_training_and_test_sets(m0,x,y,vars,lon_size,lat_size,time_period=33,dtype=dtype)

# compute ridge regressor
w_ridge = torch.zeros(lon_size*lat_size,lon_size*lat_size,dtype=dtype)
w_ridge[np.ix_(notnan_idx,notnan_idx)] = ridge_regression(x_train_merged[:,notnan_idx], y_train_merged[:,notnan_idx], lambda_=lambda_tmp, dtype=dtype)

x_test_tmp = x[m0]
y_test_tmp = y[m0]

# ridge
y_pred_ridge = prediction(x_test_tmp, w_ridge,notnan_idx, nan_idx)

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim

class Embedder(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers):
        super(Embedder, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, hidden_dim)

    def forward(self, x):
        h, _ = self.lstm(x)
        h = self.fc(h)
        return h

class Recovery(nn.Module):
    def __init__(self, hidden_dim, output_dim, num_layers):
        super(Recovery, self).__init__()
        self.lstm = nn.LSTM(hidden_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, h):
        h, _ = self.lstm(h)
        x_tilde = self.fc(h)
        return x_tilde

class Generator(nn.Module):
    def __init__(self, z_dim, hidden_dim, num_layers):
        super(Generator, self).__init__()
        self.lstm = nn.LSTM(z_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, hidden_dim)

    def forward(self, z):
        h, _ = self.lstm(z)
        h = self.fc(h)
        return h

class Supervisor(nn.Module):
    def __init__(self, hidden_dim, num_layers):
        super(Supervisor, self).__init__()
        self.lstm = nn.LSTM(hidden_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, hidden_dim)

    def forward(self, h):
        h, _ = self.lstm(h)
        h = self.fc(h)
        return h

class Discriminator(nn.Module):
    def __init__(self, hidden_dim, num_layers):
        super(Discriminator, self).__init__()
        self.lstm = nn.LSTM(hidden_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, h):
        h, _ = self.lstm(h)
        y_hat = self.fc(h)
        return y_hat

class TimeGAN(nn.Module):
    def __init__(self, input_dim, hidden_dim, z_dim, num_layers):
        super(TimeGAN, self).__init__()
        self.embedder = Embedder(input_dim, hidden_dim, num_layers)
        self.recovery = Recovery(hidden_dim, input_dim, num_layers)
        self.generator = Generator(z_dim, hidden_dim, num_layers)
        self.supervisor = Supervisor(hidden_dim, num_layers)
        self.discriminator = Discriminator(hidden_dim, num_layers)

    def forward(self, x, z):
        h = self.embedder(x)
        x_tilde = self.recovery(h)
        e_hat = self.generator(z)
        h_hat_supervise = self.supervisor(h)
        y_fake = self.discriminator(e_hat)
        y_real = self.discriminator(h)
        return x_tilde, h, e_hat, h_hat_supervise, y_fake, y_real


In [12]:
# we try with a different loss function
nbEpochs = 500
input_dim = len(notnan_idx)  # Number of features in the time series
hidden_dim = 24
z_dim = 8
num_layers = 3

z = torch.randn(x_train.shape[0], 33, z_dim)  # Latent space input

model = TimeGAN(input_dim, hidden_dim, z_dim, num_layers)
optimizer_embedder = optim.Adam(model.parameters(), lr=1e-3)
optimizer_recovery = optim.Adam(model.parameters(), lr=1e-3)
optimizer_generator = optim.Adam(model.parameters(), lr=1e-3)
optimizer_supervisor = optim.Adam(model.parameters(), lr=1e-3)
optimizer_discriminator = optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(nbEpochs):
    optimizer_embedder.zero_grad()
    optimizer_recovery.zero_grad()
    optimizer_generator.zero_grad()
    optimizer_supervisor.zero_grad()
    optimizer_discriminator.zero_grad()

    # Forward pass
    x_tilde, h, e_hat, h_hat_supervise, y_fake, y_real = model(x_train[:,:,notnan_idx], z)
    
    # Compute losses
    reconstruction_loss = nn.MSELoss()(x_tilde, y_train[:,:,notnan_idx])
    supervised_loss = nn.MSELoss()(h_hat_supervise[:, 1:, :], h[:, :-1, :])
    real_loss = nn.BCEWithLogitsLoss()(y_real, torch.ones_like(y_real))
    fake_loss = nn.BCEWithLogitsLoss()(y_fake, torch.zeros_like(y_fake))
    discriminator_loss = real_loss + fake_loss
    generator_loss = nn.BCEWithLogitsLoss()(y_fake, torch.ones_like(y_fake))
    embedding_loss = nn.MSELoss()(e_hat, h)

    # Total losses
    embedder_recovery_loss = reconstruction_loss + embedding_loss
    supervisor_loss = supervised_loss
    generator_total_loss = generator_loss + supervised_loss + embedding_loss
    discriminator_total_loss = discriminator_loss

    # Backward pass and optimization
    total_loss = embedder_recovery_loss + supervisor_loss + generator_total_loss + discriminator_total_loss
    total_loss.backward()

    # Backward pass and optimization
    # embedder_recovery_loss.backward(retain_graph=True)
    optimizer_embedder.step()
    optimizer_recovery.step()

    # supervisor_loss.backward(retain_graph=True)
    optimizer_supervisor.step()

    # generator_total_loss.backward(retain_graph=True)
    optimizer_generator.step()

    # discriminator_total_loss.backward()
    optimizer_discriminator.step()

    # print(f"Epoch {epoch + 1}, Losses: E/R: {embedder_recovery_loss.item()}, S: {supervisor_loss.item()}, G: {generator_total_loss.item()}, D: {discriminator_total_loss.item()}")
    print(f"Epoch {epoch + 1}, Loss: {total_loss.item()}")

Epoch 1, Loss: 2.6766889095306396
Epoch 2, Loss: 2.5944602489471436
Epoch 3, Loss: 2.529857635498047
Epoch 4, Loss: 2.47917103767395
Epoch 5, Loss: 2.445678949356079
Epoch 6, Loss: 2.4124755859375
Epoch 7, Loss: 2.3770041465759277
Epoch 8, Loss: 2.3494277000427246
Epoch 9, Loss: 2.341958522796631
Epoch 10, Loss: 2.3477163314819336
Epoch 11, Loss: 2.3404879570007324
Epoch 12, Loss: 2.320972204208374
Epoch 13, Loss: 2.29417085647583
Epoch 14, Loss: 2.2631773948669434
Epoch 15, Loss: 2.2344484329223633
Epoch 16, Loss: 2.212739944458008
Epoch 17, Loss: 2.196932554244995
Epoch 18, Loss: 2.1827104091644287
Epoch 19, Loss: 2.16831111907959
Epoch 20, Loss: 2.156818389892578
Epoch 21, Loss: 2.140984058380127
Epoch 22, Loss: 2.1189239025115967
Epoch 23, Loss: 2.101594924926758
Epoch 24, Loss: 2.073291778564453
Epoch 25, Loss: 2.0468266010284424
Epoch 26, Loss: 2.010625123977661
Epoch 27, Loss: 1.972022294998169
Epoch 28, Loss: 1.931427240371704
Epoch 29, Loss: 1.8952441215515137
Epoch 30, Loss: 

Epoch 82, Loss: 1.5541326999664307
Epoch 83, Loss: 1.554159164428711
Epoch 84, Loss: 1.554139256477356
Epoch 85, Loss: 1.5540242195129395
Epoch 86, Loss: 1.553523063659668
Epoch 87, Loss: 1.5526559352874756
Epoch 88, Loss: 1.5513100624084473
Epoch 89, Loss: 1.550163984298706
Epoch 90, Loss: 1.5494942665100098
Epoch 91, Loss: 1.54931640625
Epoch 92, Loss: 1.5494979619979858
Epoch 93, Loss: 1.5501124858856201
Epoch 94, Loss: 1.5511109828948975
Epoch 95, Loss: 1.551685094833374
Epoch 96, Loss: 1.5504647493362427
Epoch 97, Loss: 1.547760248184204
Epoch 98, Loss: 1.5466070175170898
Epoch 99, Loss: 1.5478262901306152
Epoch 100, Loss: 1.547975778579712
Epoch 101, Loss: 1.546574592590332
Epoch 102, Loss: 1.5452921390533447
Epoch 103, Loss: 1.5461037158966064
Epoch 104, Loss: 1.5473510026931763
Epoch 105, Loss: 1.5467283725738525
Epoch 106, Loss: 1.5445255041122437
Epoch 107, Loss: 1.5451717376708984
Epoch 108, Loss: 1.5450897216796875
Epoch 109, Loss: 1.5446579456329346
Epoch 110, Loss: 1.5434

In [13]:
y_pred_gan = model(x_test[:,:,notnan_idx], z)[0]

In [14]:
# compare the rmse of the three methods
rmse_ridge = torch.sqrt(torch.nanmean((y_test_tmp - y_pred_ridge)**2))
rmse_vae = torch.sqrt(torch.mean((y_test_tmp[:,:,notnan_idx] - y_pred_vae)**2))
rmse_gan = torch.sqrt(torch.mean((y_test_tmp[:,:,notnan_idx] - y_pred_gan)**2))

print(f"RMSE Ridge: {rmse_ridge}")
print(f"RMSE VAE: {rmse_vae}")
print(f"RMSE GAN: {rmse_gan}")

RMSE Ridge: 0.3049261271953583
RMSE VAE: 0.5482344627380371
RMSE GAN: 0.2659626603126526
